# Creación de LUT de coeficientes 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tool.ff_model  as fm
from tool._fixedInt import * 
from tool.DSPtools import *

In [2]:
clk = 100e6                 #Clock Freq.  
N = 4                       #Oversampling 
BR = clk / N                #Baud Rate 
rolloff = 0.5               
fs = N*BR                   #Sample Freq. 
Ts = 1/fs                   #Sample T. 
T = 1/BR                    #Symbol sep 
M = 2                       #Symbols (levels) 
Nsymb = 1e3                 #Simulation Length 
NB = 6                      #RC taps
PRBSn = 9                   #PRBS order

In [3]:
(t,h) = rcosine(rolloff,T,N,NB,Norm=False)
#Cuantizo 
Fp = arrayFixedInt(8,7, h,'S', 
    roundMode='round', saturateMode='saturate')
hFp = [FixedPintValue.fValue for FixedPintValue in Fp]
print(hFp)
print(Fp[12].intvalue)


[0.0, 0.0078125, 0.015625, 0.0234375, 0.0, -0.0546875, -0.1171875, -0.125, 0.0, 0.265625, 0.6015625, 0.890625, 0.9921875, 0.890625, 0.6015625, 0.265625, 0.0, -0.125, -0.1171875, -0.0546875, 0.0, 0.0234375, 0.015625, 0.0078125, 0.0]
127


In [7]:
def CreateLutVerilog(coffs, NB, N,FileName,DEC=0,intWidht=8): 
    LutFile = open(FileName,'w')
    if (not DEC):
        for mux in range(NB): 
            for coefOp in range(N):
                numCoef = coefOp + mux*N
                cofBin = bin(coffs[numCoef].intvalue)[2:]
                cofBin = cofBin.zfill(intWidht) 
                cofBin = str(intWidht)+"'b"+ str(cofBin)
                comment= "Reg{}, Mux Input {} =  h{}".format(mux,coefOp,numCoef)
                print("assign coef[{}][{}] = {}; \t /* {} */".format(mux,coefOp,cofBin,comment))
                LutFile.write("assign coef[{}][{}] = {}; \t /* {} */".format(mux,coefOp,cofBin,comment))
                LutFile.write("\n")                                                  
    
    LutFile.close()


In [8]:
CreateLutVerilog(Fp,NB,N,"./coffs.v")

assign coef[0][0] = 8'b00000000; 	 /* Reg0, Mux Input 0 =  h0 */
assign coef[0][1] = 8'b00000001; 	 /* Reg0, Mux Input 1 =  h1 */
assign coef[0][2] = 8'b00000010; 	 /* Reg0, Mux Input 2 =  h2 */
assign coef[0][3] = 8'b00000011; 	 /* Reg0, Mux Input 3 =  h3 */
assign coef[1][0] = 8'b00000000; 	 /* Reg1, Mux Input 0 =  h4 */
assign coef[1][1] = 8'b11111001; 	 /* Reg1, Mux Input 1 =  h5 */
assign coef[1][2] = 8'b11110001; 	 /* Reg1, Mux Input 2 =  h6 */
assign coef[1][3] = 8'b11110000; 	 /* Reg1, Mux Input 3 =  h7 */
assign coef[2][0] = 8'b00000000; 	 /* Reg2, Mux Input 0 =  h8 */
assign coef[2][1] = 8'b00100010; 	 /* Reg2, Mux Input 1 =  h9 */
assign coef[2][2] = 8'b01001101; 	 /* Reg2, Mux Input 2 =  h10 */
assign coef[2][3] = 8'b01110010; 	 /* Reg2, Mux Input 3 =  h11 */
assign coef[3][0] = 8'b01111111; 	 /* Reg3, Mux Input 0 =  h12 */
assign coef[3][1] = 8'b01110010; 	 /* Reg3, Mux Input 1 =  h13 */
assign coef[3][2] = 8'b01001101; 	 /* Reg3, Mux Input 2 =  h14 */
assign coef[3][3] = 